In [2]:
#%matplotlib inline
#%matplotlib qt
#%matplotlib tk
%matplotlib osx

In [3]:
import numpy as np
import scipy as sp
import scipy.ndimage as ndimage
import scipy.signal as signal
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import itertools
import random
import math
import numba
from sklearn.preprocessing import StandardScaler

In [4]:
from scipy.optimize import minimize, approx_fprime, check_grad


In [5]:
np.set_printoptions(edgeitems=25, linewidth=100) 

In [6]:
import fileio
import massage

In [7]:
import config; reload(config)
import fileio; reload(fileio)
#fileio.encode_notes(verbose=True)

<module 'fileio' from 'fileio.pyc'>

In [8]:
reload(fileio)
reload(massage)
song = massage.quantized_song(fileio.get_one_song())

In [8]:
fileio.iter_evented_songs().next()[0]

's/WALLSTRT'

In [9]:
reload(massage)
#Note that the MIDI data is quantized to 16th notes (=0.25?) by default in music21
field_predictor_list, response_list, field_pars = massage.all_base_features(song,
    quantum=0.25,
    n_pitchclasses=12,
    frequencies=(0, 1.0/4, 1.0/3, 1.0/2, 2.0/3, 1.0, 4.0/3, 3.0/2, 2.0),
    half_lives=(1, 2, 4, 8, 16)
)

In [10]:
reload(massage)
predictors, response = massage.as_trials(field_predictor_list, response_list)

In [ ]:
import fit; reload(fit)
scaler = fit.PoissonLogScaler()
predictors_sc = scaler.fit_transform(predictors)
est = fit.ExpPoissonL1Est(predictors_sc, response)

res = fit.coord_desc_reg_path(est,
    #debug=True,
    progress=True,
    norm_order=1,
    update_iter=2,
    stoch_coord=False,
    n_penalties=1024,
    update_offset=True,
)
# fileio.saveobj(res, "WALLSTRT")


In [11]:
#fileio.saveobj(res, "WALLSTRT3")
res=fileio.loadobj("WALLSTRT3")


In [ ]:
import fit; reload(fit)
scaler = fit.PoissonLogScaler()
predictors_sc = scaler.fit_transform(predictors)
est = fit.ExpPoissonL1Est(predictors_sc, response)

res2 = fit.coord_desc_reg_path(est,
    #debug=True,
    progress=True,
    norm_order=1,
    update_iter=2,
    stoch_coord=False,
    n_penalties=256,
    update_offset=True,
)
#fileio.saveobj(res2, "WALLSTRT2")


In [12]:
_=plt.semilogx(res['pen_params'],res['theta_hat_sc_path']);
plt.gca().set_yscale('symlog')
plt.ylim((-1,1))

(-1, 1)

In [15]:
_=plt.semilogx(res['pen_params'],res['theta_hat_sc_path']);


In [19]:
res.keys()

['theta_hat_sc_path',
 'logliks_total',
 'pen_params',
 'logliks_base',
 'n_nonzeros']

In [28]:
# AIC plot
_=plt.semilogx(res['pen_params'],(res['n_nonzeros']-res['logliks_base'])*2)

In [29]:
# AICc plot
_=plt.semilogx(res['pen_params'],(res['n_nonzeros']-res['logliks_base'])*2
               + 2*res['n_nonzeros']*(res['n_nonzeros']+1)/(song.shape[0]-res['n_nonzeros']-1))


In [14]:
#fileio.saveobj(res, "WALLSTRT")
#fileio.saveobj(res2, "WALLSTRT2")
res2=fileio.loadobj("WALLSTRT2")

# predictor plotting

In [10]:
scaler = StandardScaler()
fields0 = scaler.fit_transform(field_predictor_list[0])
fields1 = scaler.fit_transform(field_predictor_list[1])
with sns.axes_style("white"):
    fig, axes = plt.subplots(2,1)
    axes[0].imshow(fields0[-400:,:], interpolation='nearest')
    axes[1].imshow(fields1[-400:,:], interpolation='nearest')
    plt.show()

In [72]:
with sns.axes_style("white"):
    fig, axes = plt.subplots(2,1)
    axes[0].imshow(field_predictor_list[0][-100:,200:500], interpolation='nearest')
    axes[1].imshow(field_predictor_list[1][-100:,200:500], interpolation='nearest')
    plt.show()

In [ ]:
with sns.axes_style("white"):
    fig, ax = plt.subplots(1,1)
    ax.imshow(field_predictor_list[1][-100:,200:500], interpolation='nearest')
    plt.show()

In [74]:
field_predictor_list[0].mean(), field_predictor_list[1].mean(), field_predictor_list[2].mean()

(4.6724524166199544, 4.6724524166199535, 4.6724524166199597)

In [166]:
with sns.axes_style("white"):
    fig, ax = plt.subplots(1,1)
    ax.imshow(responses[-500:,:], interpolation='nearest')
    plt.show()